In [ ]:
!pip uninstall -y chgnet
!pip install git+https://github.com/materialsvirtuallab/chgnet.git
!pip install 'chgnet[ase]'


  Cloning https://github.com/materialsvirtuallab/chgnet.git to /tmp/pip-req-build-1brxzz9j
  Running command git clone --filter=blob:none --quiet https://github.com/materialsvirtuallab/chgnet.git /tmp/pip-req-build-1brxzz9j
  fatal: could not read Username for 'https://github.com': No such device or address
  error: subprocess-exited-with-error
  
  × git clone --filter=blob:none --quiet https://github.com/materialsvirtuallab/chgnet.git /tmp/pip-req-build-1brxzz9j did not run successfully.
  │ exit code: 128
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: subprocess-exited-with-error

× git clone --filter=blob:none --quiet https://github.com/materialsvirtuallab/chgnet.git /tmp/pip-req-build-1brxzz9j did not run successfully.
│ exit code: 128
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━

In [ ]:
import os
# Set the environment variable to reduce fragmentation before any GPU memory is allocated
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

import torch
import csv
from ase.constraints import FixAtoms
from ase.io import read
from ase.mep import NEB
from ase.optimize import BFGS, QuasiNewton
from chgnet.model.dynamics import CHGNetCalculator

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Set the base directory where your structures are stored
base_drive_path = "/content/drive/MyDrive/Dataset-1/"  # Modify as needed

device = "cuda" if torch.cuda.is_available() else "cpu"

# Get a sorted list of all subdirectories in the base directory (process all folders)
all_folders = sorted(
    [folder for folder in os.listdir(base_drive_path)
     if os.path.isdir(os.path.join(base_drive_path, folder))],
    key=lambda x: x
)

# Loop through each subdirectory
for structure_name in all_folders:
    structure_path = os.path.join(base_drive_path, structure_name)
    initial_path = os.path.join(structure_path, "initial", "POSCAR")
    final_path = os.path.join(structure_path, "final", "POSCAR")

    # Create a new folder named "CHGNET" inside the structure folder for output
    save_dir = os.path.join(structure_path, "CHGNET")
    os.makedirs(save_dir, exist_ok=True)

    # Check if energies.csv already exists, if yes, skip processing this structure
    csv_file = os.path.join(save_dir, "energies.csv")
    if os.path.exists(csv_file):
        print(f"Skipping {structure_name}: energies.csv already exists.")
        continue

    # Check if both POSCAR files exist
    if not os.path.exists(initial_path) or not os.path.exists(final_path):
        print(f"Skipping {structure_name}: Missing POSCAR in initial or final directory")
        continue

    print(f"Processing {structure_name}")
    # Read the initial and final structures
    initial = read(initial_path)
    final = read(final_path)

    # Apply constraints: fix atoms with tag > 1
    mask = [atom.tag > 1 for atom in initial]
    initial.set_constraint(FixAtoms(mask=mask))
    final.set_constraint(FixAtoms(mask=mask))

    # Assign a new CHGNet calculator for each structure endpoint
    calc_initial = CHGNetCalculator(device=device)
    calc_final = CHGNetCalculator(device=device)
    initial.calc = calc_initial
    final.calc = calc_final

    # Relax the initial and final structures (no trajectory saved)
    qn = QuasiNewton(initial)
    qn.run(fmax=0.05, steps=1000)
    qn = QuasiNewton(final)
    qn.run(fmax=0.05, steps=1000)

    # Set up NEB: create 3 intermediate images between the relaxed endpoints
    images = [initial] + [initial.copy() for _ in range(3)] + [final]
    for image in images:
        # Create a fresh calculator for each image to avoid sharing GPU memory
        image.calc = CHGNetCalculator(device=device)
        image.set_constraint(FixAtoms(mask=mask))

    neb = NEB(images, parallel=True, k=5, method='eb')
    neb.interpolate('idpp')
    qn = BFGS(neb)
    qn.run(fmax=0.05, steps=1000)

    # Compute and store energies for each image in energies.csv
    energies = []
    with open(csv_file, mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["Image Index", "Energy (eV)"])
        for i, image in enumerate(images):
            energy = image.get_potential_energy()
            energies.append(energy)
            writer.writerow([i, energy])

    # Calculate and record the migration barrier (difference between max and min energies)
    if None in energies:
        print(f"Skipping migration barrier calculation for {structure_name} due to energy errors")
    else:
        migration_barrier = max(energies) - min(energies)
        print(f"{structure_name} - Migration Barrier: {migration_barrier:.6f} eV")
        with open(csv_file, mode="a", newline="") as file:
            writer = csv.writer(file)
            writer.writerow([])
            writer.writerow(["Migration Barrier", migration_barrier])

    # Clear the GPU cache to free memory before processing the next structure
    torch.cuda.empty_cache()

print("All NEB Calculations complete!")





Streaming output truncated to the last 5000 lines.
BFGS:    4 21:17:19     -588.450378        0.285966
BFGS:    5 21:17:19     -588.457703        0.299394
BFGS:    6 21:17:20     -588.466187        0.287287
BFGS:    7 21:17:21     -588.476746        0.244273
BFGS:    8 21:17:22     -588.482971        0.197143
BFGS:    9 21:17:23     -588.490356        0.196835
BFGS:   10 21:17:23     -588.496399        0.178420
BFGS:   11 21:17:24     -588.502136        0.181992
BFGS:   12 21:17:24     -588.506470        0.147785
BFGS:   13 21:17:24     -588.510437        0.131017
BFGS:   14 21:17:25     -588.515015        0.155213
BFGS:   15 21:17:25     -588.519043        0.150162
BFGS:   16 21:17:25     -588.522583        0.111058
BFGS:   17 21:17:26     -588.524231        0.091826
BFGS:   18 21:17:26     -588.525574        0.076568
BFGS:   19 21:17:27     -588.527100        0.063312
BFGS:   20 21:17:27     -588.527893        0.086594
BFGS:   21 21:17:27     -588.528503        0.082285
BFGS:   22 21